<a href="https://colab.research.google.com/github/Ale-Coeto/vision/blob/main/NAO_ColorDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [132]:
import os
import torch
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

In [133]:
images_dir = "raw"
results_dir = "output"

if not os.path.exists(images_dir):
  os.makedirs(images_dir)

if not os.path.exists(results_dir):
  os.makedirs(results_dir)



In [201]:
# own color
starting_color = "red"

images = [str("NAO" + str(i) + ".png") for i in range(1, 7)]

for file in images:
  image = cv2.imread(os.path.join(images_dir, file))

  hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

  #0-179, 0-255 and 0-255 Hue, saturation, value
  bounds_blue = [(100, 100, 80),(120, 255, 255)]
  bounds_white = [(20, 5, 200),(120, 50, 240)]
  bounds_red = [(170, 140, 140),(179, 255, 255)]

  mask_blue = cv2.inRange(hsv_img, bounds_blue[0], bounds_blue[1])
  mask_white = cv2.inRange(hsv_img, bounds_white[0], bounds_white[1])
  mask_red = cv2.inRange(hsv_img, bounds_red[0], bounds_red[1])

  mask_blue = np.array(mask_blue)
  mask_white = np.array(mask_white)
  mask_red = np.array(mask_red)


  masked_blue = cv2.bitwise_and(image, image, mask=mask_blue)
  masked_white = cv2.bitwise_and(image, image, mask=mask_white)
  masked_red = cv2.bitwise_and(image, image, mask=mask_red)

  if (starting_color == 'red'):
    color_image = cv2.bitwise_or(masked_blue, masked_white)
  else:
    color_image = cv2.bitwise_or(masked_red, masked_white)



  blur = cv2.GaussianBlur(color_image, (0, 0), 3)
  #cv2.imwrite(os.path.join(results_dir, file), blur)

  gray = cv2.cvtColor(blur, cv2.COLOR_BGR2GRAY)

  b = cv2.cvtColor(masked_blue, cv2.COLOR_BGR2GRAY)
  w = cv2.cvtColor(masked_white, cv2.COLOR_BGR2GRAY)
  r = cv2.cvtColor(masked_red, cv2.COLOR_BGR2GRAY)

  blue_contours, _ = cv2.findContours(b, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  white_contours, _ = cv2.findContours(w, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  red_contours, _ = cv2.findContours(r, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

  contours, _ = cv2.findContours(gray, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  contours = sorted(contours, key=cv2.contourArea, reverse=True)
  largest_contour = contours[0]
  x_min = color_image.shape[0]
  x_max = 0

  flag = False
  if starting_color == 'red':
    color_contours = blue_contours
  else:
    color_contours = red_contours



  for cont in contours:
    x,y,w,h = cv2.boundingRect(cont)
    xf = x + w
    yf = y + h

    for color in color_contours:
      xb,yb,wb,hb = cv2.boundingRect(color)

      if (xb >= x and xb + wb <= xf and yb >= y and yb + hb <= yf):
        largest_contour = cont
        x_min = min(x_min,xb)
        x_max = max(x_max,xb + wb)
        flag = True

    if flag:
      break



  x,y,w,h = cv2.boundingRect(largest_contour)
  #cv2.drawContours(color_image,contours[0],-1,(0,0,205),2,cv2.LINE_AA)
  color_image = cv2.rectangle(color_image,(x_min,y),(x_max,y+h),(255,0,0),2)
  cv2.imwrite(os.path.join(results_dir, file), color_image)
